In [1]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
from tensorflow.keras.callbacks import TensorBoard
from thinkdsp import UncorrelatedUniformNoise, BrownianNoise, PinkNoise, UncorrelatedGaussianNoise
from thinkdsp import read_wave
import thinkdsp
import random
import numpy as np
import cv2
import os
import time
matplotlib.style.use('ggplot')



d:\Program Files\miniconda\envs\tf2.4\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class AudioAugmentation():
    '''
        This class is responsible for adding the noise and filters to the audio 
    '''
    def __init__(self, input_path, output_path, images_path):
        self.data_path = input_path
        self.output_path = output_path
        self.images_path = images_path

    def add_noise(self, sound, noise_kind, args):
        '''
            Adding the kind of noise choosed to the sound
        '''
        amp = args['amp']
        if noise_kind == 'UncorrelatedUniformNoise':
            noise = UncorrelatedUniformNoise(amp)
        
        elif noise_kind == 'BrownianNoise':
            noise = BrownianNoise(amp)

        elif noise_kind == 'PinkNoise':
            beta = args['beta']
            noise = PinkNoise(amp, beta)

        elif noise_kind == 'UncorrelatedGaussianNoise':
            noise = UncorrelatedGaussianNoise(amp)
        
        noise = noise.make_wave(duration=sound.duration, framerate=sound.framerate)
        result = sound + noise
        return result

    def filter(self, signal, high_pass=None, low_pass=None):
        '''
            Applying filters to the signal
        '''
        spectrum = signal.make_spectrum()
        if high_pass:
            spectrum.high_pass(high_pass)
        
        if low_pass:
            spectrum.low_pass(low_pass)

        wave = spectrum.make_wave()
        return wave
    
    def augment(self, sound):
        '''
            In this function, noises and filters are choosen randomly and added to the sound 
        '''
        noises = ['UncorrelatedUniformNoise', 'BrownianNoise', 'PinkNoise','UncorrelatedGaussianNoise']
        high_pass_value = random.randint(0,1000)
        low_pass_value = random.randint(2000, 5000)

        if random.random() > 0.3:
            sound = self.filter(sound, high_pass=high_pass_value)
        
        if random.random() > 0.3:
            sound = self.filter(sound, low_pass=low_pass_value)
        
        noise_name = random.choice(noises)
        amp = random.random()
        if noise_name == 'PinkNoise':
            beta = random.random()
            sound = self.add_noise(sound, noise_name, {'amp':amp, 'beta': beta})
        else:
            sound = self.add_noise(sound, noise_name, {'amp':amp})
        return sound    

    def augment_dataset(self, n_augmentations):
        '''
            Iterating over the data and applying the augmentations
        '''
        for file in os.listdir(self.data_path):
            sound = read_wave(rf'{self.data_path}\{file}')
            for i in range(n_augmentations):
                sound_augmented = self.augment(sound)
                sound_augmented.write(rf'{self.output_path}\{i}_{file}')
            sound.write(rf'{self.output_path}\{file}')

    def save_spectograms(self, class_name):
        '''
            Saving the spectograms
        '''
        counter = 0
        for file in os.listdir(self.output_path):
            sound = read_wave(rf'{self.output_path}\{file}')
            sp = sound.make_spectrogram(256)
            sp.plot()
            plt.axis("off")
            plt.savefig(rf'{self.images_path}\n{class_name}{counter}.jpg', bbox_inches='tight', pad_inches=0)
            plt.clf()
            counter += 1
        

## Augmenting Barbie files

In [7]:
input_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie'
output_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented'
images_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\training\n0'
augmenter = AudioAugmentation(input_path, output_path, images_path)
augmenter.augment_dataset(n_augmentations=4)
augmenter.save_spectograms(class_name=0)

Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\0_barbie_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\1_barbie_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\2_barbie_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\3_barbie_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\barbie_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\0_barbie_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\1_barbie_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\2_barbie_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\barbie augmented\3_barbie_10.wav

<Figure size 640x480 with 0 Axes>

## Augmenting Puppy files

In [8]:
input_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy'
output_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented'
images_path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\training\n1'
augmenter = AudioAugmentation(input_path, output_path, images_path)
augmenter.augment_dataset(4) 
augmenter.save_spectograms(1)

Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\0_puppy_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\1_puppy_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\2_puppy_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\3_puppy_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\puppy_1.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\0_puppy_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\1_puppy_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\2_puppy_10.wav
Writing D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\puppy augmented\3_puppy_10.wav
Writing D:\Univer

<Figure size 640x480 with 0 Axes>

## Spliting the data into the training and validation files

In [9]:
path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\training\n0'
outpath = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\validation\n0'
val_imgs = []
for i in range(40):
    file_n = random.randint(0,199)
    img = cv2.imread(rf'{path}\n0{file_n}.jpg')
    val_imgs.append(rf'{path}\n0{file_n}.jpg')
    cv2.imwrite(rf'{outpath}\n0{file_n}.jpg', img)

for img in val_imgs:
    try:
        os.remove(img)
    except:
        pass

In [10]:
path = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\training\n1'
outpath = r'D:\University\Third year\semester 2\DSP\Assignments\A2\barbie_vs_puppy\validation\n1'
val_imgs = []
for i in range(38):
    file_n = random.randint(0,190)
    print(rf'n1{file_n}.jpg')
    img = cv2.imread(rf'{path}\n1{file_n}.jpg')
    val_imgs.append(rf'{path}\n1{file_n}.jpg')
    cv2.imwrite(rf'{outpath}\n1{file_n}.jpg', img)

for img in val_imgs:
    try:
        os.remove(img)
    except:
        pass

n127.jpg
n1166.jpg
n150.jpg
n1106.jpg
n12.jpg
n136.jpg
n1178.jpg
n1161.jpg
n11.jpg
n1109.jpg
n1103.jpg
n1156.jpg
n183.jpg
n123.jpg
n1124.jpg
n128.jpg
n14.jpg
n138.jpg
n1167.jpg
n1119.jpg
n1128.jpg
n170.jpg
n1136.jpg
n156.jpg
n171.jpg
n191.jpg
n149.jpg
n151.jpg
n174.jpg
n189.jpg
n15.jpg
n132.jpg
n1116.jpg
n125.jpg
n1139.jpg
n1161.jpg
n1124.jpg
n1143.jpg
